# Noise Model Exploration
Interactively explore the noise conversions: F→p_dep, T1/T2→decoherence, and T2*→σ for quasi-static noise.

In [ ]:
# Setup and formulas
import sys, numpy as np, math
from pathlib import Path
root = Path.cwd()
src_path = root / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))
from IPython.display import display
import ipywidgets as widgets
def p_dep_1q(F1):
    return 2.0 * (1.0 - F1)
def p_dep_2q(F2):
    return (4.0/3.0) * (1.0 - F2)
def decoherence_probs(tau, T1, T2):
    p_amp = 1.0 - math.exp(-tau / T1)
    Tphi_inv = (1.0/T2) - (1.0/(2.0*T1))
    Tphi = 1.0 / Tphi_inv if Tphi_inv > 0 else float('inf')
    p_phi = 1.0 - math.exp(-tau / Tphi) if math.isfinite(Tphi) else 0.0
    return p_amp, p_phi
def sigma_from_T2_star(T2s):
    return math.sqrt(2.0) / T2s

In [ ]:
# Widgets and interactivity
F1 = widgets.FloatSlider(value=0.99926, min=0.98, max=1.0, step=1e-5, description='F1')
F2 = widgets.FloatSlider(value=0.99800, min=0.95, max=1.0, step=1e-5, description='F2')
T1 = widgets.FloatLogSlider(value=1.0, base=10, min=-2, max=1, step=0.01, description='T1 [s]')
T2 = widgets.FloatLogSlider(value=99e-6, base=10, min=-6, max=-3, step=0.01, description='T2 [s]')
T2s = widgets.FloatLogSlider(value=20e-6, base=10, min=-6, max=-3, step=0.01, description='T2* [s]')
tau1 = widgets.FloatLogSlider(value=60e-9, base=10, min=-9, max=-6, step=0.01, description='t1 [s]')
tau2 = widgets.FloatLogSlider(value=40e-9, base=10, min=-9, max=-6, step=0.01, description='t2 [s]')
out = widgets.Output()
def recompute(_=None):
    with out:
        out.clear_output()
        print("p_dep (1q):", p_dep_1q(F1.value))
        print("p_dep (2q):", p_dep_2q(F2.value))
        p_amp1, p_phi1 = decoherence_probs(tau1.value, T1.value, T2.value)
        p_amp2, p_phi2 = decoherence_probs(tau2.value, T1.value, T2.value)
        print(f"1q gate (tau={tau1.value:.1e}s): p_amp={p_amp1:.3e}, p_phi={p_phi1:.3e}")
        print(f"2q gate (tau={tau2.value:.1e}s): p_amp={p_amp2:.3e}, p_phi={p_phi2:.3e}")
        print("sigma (quasi-static):", sigma_from_T2_star(T2s.value))
for w in [F1,F2,T1,T2,T2s,tau1,tau2]:
    w.observe(recompute, names='value')
display(widgets.VBox([F1, F2, T1, T2, T2s, tau1, tau2, out]))
recompute()